<a href="https://colab.research.google.com/github/vbonato/cnnTestBench/blob/main/cnnTestBench.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
# Updated CNN Code Generator (SRC/BIN Structure)

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
import os
import math

# --- 1. Core Code Generation Functions (Updated for Paths) ---

def generate_makefile_code(num_layers):
    """
    Generates the Makefile content.
    - Source files are in the current directory (src/).
    - Executable and output are placed in the sibling directory ../bin/.
    """
    TARGET = f'conv{num_layers}'
    TB_OBJ = f'conv_tb{num_layers}.o'
    TB_SRC = f'conv_tb{num_layers}.cpp'
    TB_HDR = f'conv_tb{num_layers}.h'
    CNN_HDR = f'conv{num_layers}.h'
    CNN_SRC = f'conv{num_layers}.cpp'

    makefile_lines = [
        # The ultimate target: creates dir, links objects, and moves executable
        f'{TARGET}: check_dirs {TARGET}.o {TB_OBJ}',
        # Compile and move the executable one directory up into the bin folder
        f'\tclang {TARGET}.o {TB_OBJ} -o ../bin/{TARGET}\n',

        # Rule for the Testbench Object File
        f'{TB_OBJ}: {TB_SRC} {TB_HDR} {CNN_HDR}',
        f'\tclang -c {TB_SRC} -o {TB_OBJ}\n',

        # Rule for the CNN Object File
        f'{TARGET}.o: {CNN_SRC} {CNN_HDR}',
        f'\tclang -c {CNN_SRC} -o {TARGET}.o\n',

        # NEW TARGET: Checks and creates the sibling bin/ folder (one level up)
        'check_dirs:',
        '\t@mkdir -p ../bin\n',

        '.PHONY: clean',
        'clean:',
        # Delete object files from the current folder (src/)
        f'\trm -f {TARGET}.o {TB_OBJ}',
        # Delete the final executable and output file from the sibling bin/ folder
        f'\trm -f ../bin/{TARGET}',
        f'\trm -f ../bin/output.bin\n'
    ]
    return '\n'.join(makefile_lines)

# --- ADDED MISSING FUNCTION (Same as before) ---
def generate_testbench_header_code(num_layers):
    """Generates the testbench header C++ code (conv_tbX.h)."""
    # Defines constants needed only by the testbench, like RANDROOF.
    code = f"#ifndef CONV_TB_H\n#define CONV_TB_H\n\n"
    code += f"// ** GLOBAL TESTBENCH CONSTANTS **\n"
    code += f"const int RANDROOF = 256; // Max value for randomized input/weights\n"
    code += f"\n#endif"
    return code
# ------------------------------

def generate_testbench_code(num_layers, params):
    """
    Generates the testbench C++ code, updated to save 'output.bin'
    to the sibling bin/ folder (../bin/output.bin).
    """
    N_CLASSES = params['N_CLASSES']

    code_lines = [
        '#include <cstdio>',
        '#include <cstdlib>',
        f'#include "conv_tb{num_layers}.h"',
        f'#include "conv{num_layers}.h"\n',
        'int main(void) {'
    ]

    # Buffers - Note: C_N, H_N, E_N refer to the Nth CONV layer
    code_lines.append(f' \t// Input Buffer')
    code_lines.append(f' \ttype_t *I1 = (type_t *) malloc(C1 * H1 * H1 * sizeof(type_t));')
    code_lines.append(f' \t// Weight Buffers for {num_layers} Conv Layers')
    for i in range(1, num_layers + 1):
        # W size: M_i * C_i * R_i * R_i
        code_lines.append(f' \ttype_t *W{i} = (type_t *) malloc(M{i} * C{i} * R{i} * R{i} * sizeof(type_t));')

    # FC Weight Buffer: size of previous output (M_N * E_N * E_N) * N_CLASSES
    code_lines.append(f' \t// Weight Buffer for Final FC Layer')
    code_lines.append(f' \ttype_t *W_fc = (type_t *) malloc((M{num_layers} * E{num_layers} * E{num_layers}) * N_CLASSES * sizeof(type_t));')

    # Output Buffer - N_CLASSES
    code_lines.append(f' \t// Final Output Buffer (N_CLASSES={N_CLASSES})')
    code_lines.append(f' \ttype_t *O_final = (type_t *) calloc(N_CLASSES, sizeof(type_t));')

    # Init inputs
    code_lines.append('\n \tsrand(1);')
    # Init Input Feature Map I1
    code_lines.append(f' \t// Initialize Input I1')
    code_lines.append(' \tfor(unsigned j = 0; j < C1 * H1 * H1; j++)')
    code_lines.append(' \t\tI1[j] = rand() % RANDROOF;')
    # Init Weights W_i
    code_lines.append(f'\n \t// Initialize Conv Weights')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tfor(unsigned j = 0; j < M{i} * C{i} * R{i} * R{i}; j++)')
        code_lines.append(f' \t\tW{i}[j] = rand() % RANDROOF;')

    # Init FC Weights
    code_lines.append(f'\n \t// Initialize FC Weights')
    code_lines.append(f' \tfor(unsigned j = 0; j < (M{num_layers} * E{num_layers} * E{num_layers}) * N_CLASSES; j++)')
    code_lines.append(f' \t\tW_fc[j] = rand() % RANDROOF;')


    # CNN Call
    code_lines.append('\n \t// Perform CNN inference')
    cnn_call = ' \tcnn(I1'
    for i in range(1, num_layers + 1):
        cnn_call += f', W{i}'
    cnn_call += f', W_fc, O_final);'
    code_lines.append(cnn_call)

    # Print output
    code_lines.append(f'\n \t// Print and save final output (N_CLASSES)')
    code_lines.append(f' \tfor(int j = 0; j < N_CLASSES; j++) {{')
    code_lines.append(f' \t\tprintf("%x ", O_final[j]);')
    code_lines.append(' \t}')
    code_lines.append(' \tprintf("\\n");\n')

    # Output to file - UPDATED PATH (../bin/ is one level up from src/)
    code_lines.append(f' \tFILE *opf = fopen("../bin/output.bin", "wb");')
    code_lines.append(f' \tfwrite(O_final, sizeof(type_t), N_CLASSES, opf);')
    code_lines.append(' \tfclose(opf);')

    # Free
    code_lines.append('\n \t// Free allocated memory')
    code_lines.append(' \tif(I1) free(I1);')
    for i in range(1, num_layers + 1):
        code_lines.append(f' \tif(W{i}) free(W{i});')
    code_lines.append(' \tif(W_fc) free(W_fc);')
    code_lines.append(' \tif(O_final) free(O_final);')

    code_lines.append('\n \treturn EXIT_SUCCESS;\n}')
    return '\n'.join(code_lines)

# (All other functions from the previous versions remain unchanged:
# generate_conv_code, generate_convh_code, calculate_output_size, generate_parameter_widgets,
# collect_and_calculate_params, update_widgets)

def generate_conv_code(num_layers):
    """Generates the convolution and wrapper C++ implementation code."""
    code = f'#include "conv{num_layers}.h"\n\n'

    # Add ReLU function
    code += "type_t relu(type_t x) {\n"
    code += " \treturn (x > 0) ? x : 0;\n"
    code += "}\n\n"

    # Generate N standard convolution layer functions (conv1 to convN)
    for i in range(1, num_layers + 1):
        # Optimized loop order for HLS pipelining (m, y, x are output loops)
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t O{i}[M{i} * E{i} * E{i}]) {{\n"
        code += f" \tfor(int m = 0; m < M{i}; m++) {{\n"
        code += f" \t\tfor(int y = 0; y < E{i}; y++) {{\n"
        code += f" \t\t\tfor(int x = 0; x < E{i}; x++) {{\n"

        # HLS Pipelining for the inner MAC operation
        code += f" \t\t\t\t#pragma HLS PIPELINE II=1\n"

        code += f" \t\t\t\tfor(int c = 0; c < C{i}; c++) {{\n" # Channel loop
        code += f" \t\t\t\t\tfor(int k = 0; k < R{i}; k++) {{\n" # Kernel height loop
        code += f" \t\t\t\t\t\tfor(int l = 0; l < R{i}; l++) {{\n" # Kernel width loop

        code += f" \t\t\t\t\t\t\t// Zero-padding implementation\n"
        code += f" \t\t\t\t\t\t\tint h1 = y * S{i} - P{i} + k;\n"
        code += f" \t\t\t\t\t\t\tint h2 = x * S{i} - P{i} + l;\n"
        code += f" \t\t\t\t\t\t\ttype_t val = (h1 < 0 || h1 >= H{i} || h2 < 0 || h2 >= H{i}) ? 0 : I{i}[h2 + (h1 + (c * H{i})) * H{i}];\n"
        code += f" \t\t\t\t\t\t\tO{i}[x + (y + (m * E{i})) * E{i}] += val * W{i}[l + (k + (c + (m * C{i})) * R{i}) * R{i}];\n"

        # FIX: Corrected closing braces
        code += f" \t\t\t\t\t\t}}\n" # Close l
        code += f" \t\t\t\t\t}}\n" # Close k
        code += f" \t\t\t\t}}\n" # Close c
        code += f" \t\t\t}}\n" # Close x
        code += f" \t\t}}\n" # Close y
        code += f" \t}}\n" # Close m
        code += f"}}\n\n"

    # Final Dedicated FC layer for classification
    input_size_n = f"M{num_layers} * E{num_layers} * E{num_layers}"

    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t output[N_CLASSES]) {{\n"
    code += f" \t// Flatten the input and perform N_CLASSES dot products\n"
    code += f" \tfor (int k = 0; k < N_CLASSES; k++) {{\n"
    code += f" \t\toutput[k] = 0;\n"
    code += f" \t\tfor (int i = 0; i < {input_size_n}; i++) {{\n"
    code += f" \t\t\t// W_fc is indexed by [output_class][input_feature]\n"
    code += f" \t\t\toutput[k] += input[i] * W_fc[i + k * {input_size_n}];\n"
    code += f" \t\t}}\n"
    code += f" \t\toutput[k] = relu(output[k]); // Apply ReLU to FC output (common practice)\n"
    code += f" \t}}\n"
    code += f"}}\n\n"

    # Wrapper CNN function
    code += "void cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}"
    code += ", type_t *W_fc, type_t *output) {\n"
    code += "#pragma HLS DATAFLOW\n"

    # Intermediate buffers for dataflow
    for i in range(1, num_layers + 1):
        code += f" \tstatic type_t O{i}[M{i} * E{i} * E{i}];\n"

    code += "\n"

    # Chaining the N convolution layers
    for i in range(1, num_layers + 1):
        if i == 1:
            # First layer uses I1 (input) and outputs to O1
            code += f" \tconv1(input, W1, O1);\n"
            code += f" \t// Apply ReLU to intermediate output O1\n"
            code += f" \tfor (int j = 0; j < M1 * E1 * E1; j++) O1[j] = relu(O1[j]);\n"
        else:
            # Intermediate layers: input O_{i-1}, output O_i
            code += f" \tconv{i}(O{i-1}, W{i}, O{i});\n"
            code += f" \t// Apply ReLU to intermediate output O{i}\n"
            code += f" \tfor (int j = 0; j < M{i} * E{i} * E{i}; j++) O{i}[j] = relu(O{i}[j]);\n"

    # Final FC layer
    code += "\n \t// Final Layer: Fully Connected (FC) Classification Head\n"
    code += f" \tfc_layer(O{num_layers}, W_fc, output);\n"

    code += "}\n"
    return code


def generate_convh_code(num_layers, params):
    """Generates the convolution header C++ code with calculated parameters."""
    code = "#ifndef CONV_H\n#define CONV_H\n\n#include <cstddef>\ntypedef unsigned type_t;\n\n"

    # Global Parameters
    code += f"// ** CLASSIFICATION PARAMETERS **\n"
    code += f"const size_t N_CLASSES = {params['N_CLASSES']};\n\n"

    # Input Parameters (Defined only once)
    code += "// ** INPUT PARAMETERS **\n"
    code += f"const size_t C1 = {params['C1']}; // Input channels\n"
    code += f"const size_t H1 = {params['H1']}; // Input size H x H\n\n"

    for i in range(1, num_layers + 1):
        code += f"// ** CONV LAYER {i} **\n"

        # FIX: Only define C_i and H_i for i > 1, as C1 and H1 are defined above.
        if i > 1:
            code += f"const size_t C{i} = {params[f'C{i}']};\n" # Input Channels of current layer
            code += f"const size_t H{i} = {params[f'H{i}']};\n" # Input Size of current layer
        else:
            # For i=1, C1 and H1 are already defined, so skip them here.
            pass

        code += f"const size_t M{i} = {params[f'M{i}']};\n" # Output Channels
        code += f"const size_t R{i} = {params[f'R{i}']};\n" # Kernel Size
        code += f"const size_t S{i} = {params[f'S{i}']};\n" # Stride
        code += f"const size_t E{i} = {params[f'E{i}']};\n" # Output Size

        # Calculated parameters
        code += f"const size_t F{i} = ((E{i} * S{i} + R{i} - 1) < H{i}) ? H{i} : (E{i} * S{i} + R{i} - 1);\n"
        code += f"const size_t P{i} = (F{i} - H{i}) / 2;\n\n"

    # Function prototypes
    for i in range(1, num_layers + 1): # Prototypes for all N convolution layers
        code += f"void conv{i}(type_t I{i}[C{i} * H{i} * H{i}], type_t W{i}[M{i} * C{i} * R{i} * R{i}], type_t O{i}[M{i} * E{i} * E{i}]);\n"

    input_size_n = f"M{num_layers} * E{num_layers} * E{num_layers}"
    code += f"void fc_layer(type_t input[{input_size_n}], type_t W_fc[{input_size_n} * N_CLASSES], type_t output[N_CLASSES]);\n"

    code += "\nvoid cnn(type_t *input"
    for i in range(1, num_layers + 1):
        code += f", type_t *W{i}"
    code += ", type_t *W_fc, type_t *output);\n" # Added W_fc

    code += "\n#endif"
    return code


# --- 2. Parameter Calculation and GUI Setup (Unchanged) ---

def calculate_output_size(H_in, R, S, P):
    """Calculates output size E based on HLS-style padding/stride. We enforce E = ceil(H_in / S)"""
    return math.ceil(H_in / S)

def generate_parameter_widgets(num_layers):
    """Generates the main parameter input widgets."""

    # Global Classification Parameter
    n_classes_widget = widgets.IntText(value=10, description='N_CLASSES:', min=1, style={'description_width': 'initial'})

    # Input Feature Map (Layer 1 Input)
    c1_widget = widgets.IntText(value=3, description='Input C1:', min=1, style={'description_width': 'initial'})
    h1_widget = widgets.IntText(value=32, description='Input H1:', min=1, style={'description_width': 'initial'})

    widgets_list = [
        widgets.HTML(value="<h3>Classification Parameters:</h3>"),
        n_classes_widget,
        widgets.HTML(value="<h3>Input Feature Map (Layer 1 Input):</h3>"),
        c1_widget,
        h1_widget,
        widgets.HTML(value=f"<h3>Convolution Layers (1 to {num_layers}):</h3>")
    ]

    layer_widgets = {}

    for i in range(1, num_layers + 1):
        # Channels Out (M_i) - user defines
        m_i = widgets.IntText(value=32, description=f'L{i} M (Ch Out):', min=1, style={'description_width': 'initial'})

        # Kernel Size (R_i) - user defines
        r_i = widgets.IntText(value=3, description=f'L{i} R (Kernel):', min=1, style={'description_width': 'initial'})

        # Stride (S_i) - user defines
        s_i = widgets.IntText(value=1, description=f'L{i} S (Stride):', min=1, style={'description_width': 'initial'})

        layer_widgets[i] = (m_i, r_i, s_i)

        widgets_list.append(widgets.VBox([
            widgets.HTML(value=f"<h4>Conv Layer {i}</h4>"),
            widgets.HBox([m_i, r_i, s_i])
        ]))

    # Wrap parameter widgets in a VBox container
    params_vbox = widgets.VBox(widgets_list)
    return params_vbox, layer_widgets, c1_widget, h1_widget, n_classes_widget

def collect_and_calculate_params(num_layers, layer_widgets, c1_widget, h1_widget, n_classes_widget):
    """Collects user input and calculates dependent parameters (C_i, H_i, E_i)."""

    params = {}

    # Global Parameter
    params['N_CLASSES'] = n_classes_widget.value

    # Initial Input
    params['C1'] = c1_widget.value
    params['H1'] = h1_widget.value

    H_prev = params['H1']
    M_prev = params['C1']

    for i in range(1, num_layers + 1):
        m_i, r_i, s_i = layer_widgets[i]

        # C_i (Input Channels) = M_{i-1} (Output Channels of prev layer)
        # H_i (Input Size) = E_{i-1} (Output Size of prev layer)
        params[f'C{i}'] = M_prev
        params[f'H{i}'] = H_prev

        # User Defined Parameters
        params[f'M{i}'] = m_i.value
        params[f'R{i}'] = r_i.value
        params[f'S{i}'] = s_i.value

        # Calculated Output Size (E_i) - Enforcing 'Same' or 'Half' Padding
        params[f'E{i}'] = calculate_output_size(params[f'H{i}'], params[f'R{i}'], params[f'S{i}'], 0) # P is calculated later

        # For next layer's input
        H_prev = params[f'E{i}']
        M_prev = params[f'M{i}']

        # Validation
        if params[f'R{i}'] > params[f'H{i}'] and params[f'H{i}'] > 1:
            raise ValueError(f"Layer {i}: Kernel size R{i}={params[f'R{i}']} must be less than or equal to input size H{i}={params[f'H{i}']}.")
        if params['N_CLASSES'] < 1:
             raise ValueError("N_CLASSES must be 1 or greater.")

    return params


# --- 3. GUI and Execution (Updated Folder Logic) ---

layer_slider = widgets.IntSlider(value=3, min=1, max=5, step=1, description='CNN Layers:', continuous_update=False, style={'description_width': 'initial'})
generate_button = widgets.Button(description='Generate Code', button_style='success')
output_area = widgets.Output()
code_controls_vbox = widgets.VBox() # Container for the dynamically changing parameter widgets

# Initial setup of parameter widgets
param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input = generate_parameter_widgets(layer_slider.value)
code_controls_vbox.children = (param_vbox,) # Place generated widgets into the container

def update_widgets(change):
    """Update parameter widgets when the number of layers changes."""
    global layer_widgets_map, c1_input, h1_input, n_classes_input

    num_layers = layer_slider.value
    # Generate a new set of widgets
    new_param_vbox, layer_widgets_map, c1_input, h1_input, n_classes_input = generate_parameter_widgets(num_layers)

    # Replace the content of the VBox container
    code_controls_vbox.children = (new_param_vbox,)

    with output_area:
          clear_output()
          display(Markdown(f"Parameters updated for **{num_layers} layers**. Click 'Generate Code'."))

layer_slider.observe(update_widgets, names='value')

def on_button_click(b):
    """Code generation main logic. Updates paths to use SRC/BIN structure."""
    with output_area:
        clear_output()

        num_layers = layer_slider.value
        i = num_layers

        try:
            # 1. Collect and Calculate Parameters
            params = collect_and_calculate_params(num_layers, layer_widgets_map, c1_input, h1_input, n_classes_input)

            # 2. Setup Folder (Dynamically named based on number of layers)
            # Source folder is now nested: ./repo/generatedCNN_NLayers/src
            root_folder = f'./repo/generatedCNN_{i}Layers'
            source_folder = os.path.join(root_folder, 'src')
            bin_folder = os.path.join(root_folder, 'bin') # Ensure bin folder is created
            os.makedirs(source_folder, exist_ok=True)
            os.makedirs(bin_folder, exist_ok=True) # Ensure bin folder is also created by Python for clean setup

            # 3. Generate Files

            # All files are written into the 'src' subfolder
            with open(f'{source_folder}/Makefile{i}', 'w') as f: f.write(generate_makefile_code(i))
            with open(f'{source_folder}/conv_tb{i}.cpp', 'w') as f: f.write(generate_testbench_code(i, params))
            with open(f'{source_folder}/conv_tb{i}.h', 'w') as f: f.write(generate_testbench_header_code(i))
            with open(f'{source_folder}/conv{i}.h', 'w') as f: f.write(generate_convh_code(i, params))
            with open(f'{source_folder}/conv{i}.cpp', 'w') as f: f.write(generate_conv_code(i))

            display(Markdown(f'## ✅ Success! All files for **{i} CONV layers** + **1 FC layer** generated in `{source_folder}`'))

            # Print a summary of the calculated parameters
            param_summary = ["| Layer | Type | C (In) | H (In) | M (Out) | R (K) | S (Str) | E (Out) |"]
            param_summary.append("|---|---|---|---|---|---|---|---|")

            # Initial Input
            param_summary.append(f"| Input | Image | {params['C1']} | {params['H1']} | N/A | N/A | N/A | N/A |")

            for j in range(1, num_layers + 1):
                param_summary.append(f"| {j} | CONV | {params[f'C{j}']} | {params[f'H{j}']} | {params[f'M{j}']} | {params[f'R{j}']} | {params[f'S{j}']} | {params[f'E{j}']} |")

            # Final FC layer summary
            input_size = params[f'M{num_layers}'] * params[f'E{num_layers}'] * params[f'E{num_layers}']
            param_summary.append(f"| N+1 | **FC** | {input_size} (flat) | N/A | **{params['N_CLASSES']}** | N/A | N/A | N/A |")

            display(Markdown('### Calculated CNN Parameters\n' + '\n'.join(param_summary)))

            display(Markdown('### Execution Instructions (SRC/BIN Structure)'))
            display(Markdown(
                f"1. **Navigate to the Source:** `cd {source_folder}`\n"
                f"2. **Compile:** `make -f Makefile{i} conv{i}`\n"
                f"3. **Run:** Go to the parent directory: `cd ../` and execute the binary: `./bin/conv{i}`\n"
                f"The executable (`conv{i}`) and the output file (`output.bin`) are now in the **`{root_folder}/bin/`** folder."
            ))

        except ValueError as e:
            display(Markdown(f'## ❌ Error: {e}'))

        except Exception as e:
            display(Markdown(f'## ❌ An unexpected error occurred: {e}'))


generate_button.on_click(on_button_click)

# FINAL DISPLAY: Display the components only once
display(layer_slider, code_controls_vbox, generate_button, output_area)

IntSlider(value=3, continuous_update=False, description='CNN Layers:', max=5, min=1, style=SliderStyle(descrip…

Button(button_style='success', description='Generate Code', style=ButtonStyle())

Output()